In [1]:
%autosave 45
#!/usr/bin/python
# -*- coding: utf-8 -*-

import tweepy
import spacy
import csv
import json
import pandas as pd
import numpy as py
import os

Autosaving every 45 seconds


In [2]:
# Twitter API credentials
credentialsPath = r'..\0_data\credentials'
with open(os.path.join(credentialsPath, 'twitter_credentials.json')) as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

# Create the api endpoint

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [3]:
# Convert to data frame
def toDataFrame(tweets):
    DataSet = pd.DataFrame()
    
    # Get tweet information
    tweetIDs = []
    for tweet in tweets:
        tweetIDs.append(tweet.id)              
    DataSet['Tweet ID'] = [ID for ID in tweetIDs]
        
    tweetsText = []
    for tweet in tweets:
            if 'retweeted_status' in  dir(tweet):
                tweetsText.append(tweet.retweeted_status.full_text.encode('utf-8'))                
            else:
                tweetsText.append(tweet.full_text.encode('utf-8'))  
    DataSet['Text'] = [text for text in tweetsText]
    
    # Get user information
    DataSet['User'] = [tweet.user.name.encode('utf-8') for tweet in tweets]
    
    DataSet['User ID'] = [tweet.user.id for tweet in tweets]
    
    # Get media
    tweetsImages = []
    for tweet in tweets:
        if 'media' in tweet.entities:
            for image in tweet.entities['media']:
                tweetsImages.append(image['media_url'])
        else:
            tweetsImages.append('')  
    DataSet['Image Urls'] = [image for image in tweetsImages]
          
    # Get location    
    tweetsLongitudes = []
    for tweet in tweets:
        if tweet.coordinates is not None:
            tweetsLongitudes.append(tweet.coordinates["coordinates"][0])
        else:
            tweetsLongitudes.append('')
    DataSet['Longitude'] = [longitude for longitude in tweetsLongitudes]
    
    tweetsLatitudes = []
    for tweet in tweets:
        if tweet.coordinates is not None:
            tweetsLatitudes.append(tweet.coordinates["coordinates"][1])
        else:
            tweetsLatitudes.append('')
    DataSet['Latitude'] = [latitude for latitude in tweetsLatitudes]
    
    locations = []
    for tweet in tweets:
        if tweet.user.location is not None:
            locations.append(tweet.user.location.encode('utf-8'))
        else:
            locations.append(np.nan)            
    DataSet['Location'] = [loc for loc in locations]
    
    # Get other fields  
    tweetsPosted = []
    for tweet in tweets:
        tweetsPosted.append(tweet.created_at)
    DataSet['Created'] = [created for created in tweetsPosted]
    
    return DataSet

# Specify the term you want to scrape
mention = input('Enter the term you want to scrape- ')

results = []

for tweet_info in tweepy.Cursor(api.search, q=mention,
                           tweet_mode='extended').items():
    results.append(tweet_info)

dataframe = toDataFrame(results)

Enter the term you want to scrape- @sfmta_muni


In [4]:
# Load the spacy library and English corpus
nlp = spacy.load('en')

nlp.pipeline

[('tagger', <spacy.pipeline.Tagger at 0x1c00539c400>),
 ('parser', <spacy.pipeline.DependencyParser at 0x1c00545fc50>),
 ('ner', <spacy.pipeline.EntityRecognizer at 0x1c00545fca8>)]

In [5]:
from collections import defaultdict

# Instantiate cleaned data dictionary
cleaned_data = defaultdict(list)

# Clean the text data
for row in dataframe.itertuples():
    tweetid = row[1]
    text = row[2].decode('utf-8').encode('ascii', 'ignore').decode('ascii').strip()
    user = row[3].decode('utf-8').encode('ascii', 'ignore').decode('ascii').strip()
    userid = row[4]
    imageurl = row[5]
    longitude = row[6]
    latitude = row[7]
    location = row[8]
    created = row[9]
    
    text_doc = nlp.make_doc(text)
    tokens = [token for token in text_doc]
    tokens = [token for token in tokens if not token.is_punct]
    tokens = [token for token in tokens if token.text != ' ']    
    tokens_final = [token.text for token in tokens]
    
    text = " ".join(tokens_final).strip()
    text = text.replace('\n', '')
    
    cleaned_data['Tweet ID'].append(tweetid)
    cleaned_data['Text'].append(text)
    cleaned_data['User'].append(user)
    cleaned_data['User ID'].append(userid)
    cleaned_data['Image'].append(imageurl)
    cleaned_data['Longitude'].append(longitude)
    cleaned_data['Latitude'].append(latitude)
    cleaned_data['Location'].append(location)
    cleaned_data['Created'].append(created)

cleaned_dataframe = pd.DataFrame(cleaned_data)

In [6]:
outputPath = r'..\0_data\manual'
filePath = os.path.join(outputPath,'tweets_with_' + 'mention_' + mention + '.csv')
if not os.path.isfile(filePath):
    cleaned_dataframe.to_csv(filePath, index=False)
else:
    with open(filePath, 'a') as file:
        cleaned_dataframe.to_csv(file, index = False)
print ('Extracted ' + str(len(results)) 
    + ' tweets with ' + mention)

Extracted 2125 tweets with @sfmta_muni
